In [1]:
# Importação de bibliotecas necessárias
import pandas as pd
import csv
import requests
import numpy as np
from io import StringIO
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. URL do CSV
url = "https://raw.githubusercontent.com/ericasilva1994/ibovespa-dataset/refs/heads/main/Planilha%20inovespa%20%20desde%201982%20-%20P%C3%A1gina1.csv"

# 2. Baixa o conteúdo como texto
resposta = requests.get(url)
conteudo = resposta.text

# 3. Usa StringIO para simular um arquivo
f = StringIO(conteudo)

# 4. Processa com csv.reader (respeita aspas e vírgulas)
leitor = csv.reader(f, delimiter=',', quotechar='"')
linhas = list(leitor)

# 5. Cria DataFrame com os dados
df_raw = pd.DataFrame(linhas)

# 6. Divide a string da primeira coluna por vírgula (se necessário)
df_split = df_raw[0].str.split(',', expand=True)

# 7. Renomeia e limpa
df_split = df_split.iloc[:, :7]
df_split.columns = ['Data', 'Último', 'Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%']
df_split = df_split.applymap(lambda x: x.strip().replace('"', '') if isinstance(x, str) else x)

df = df_split
df.head()


/tmp/ipython-input-1-2746274536.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_split = df_split.applymap(lambda x: x.strip().replace('"', '') if isinstance(x, str) else x)


,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
1,15.05.2002,12.350,12.207,12.468,12.114,313,27M
2,14.05.2002,12.204,11.999,12.327,11.999,317,39M
3,13.05.2002,12.002,12.147,12.212,11.943,145,04M
4,10.05.2002,12.130,12.106,12.180,11.944,282,86M


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10747 entries, 0 to 10746
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Data      10747 non-null  object
 1   Último    10747 non-null  object
 2   Abertura  10747 non-null  object
 3   Máxima    10747 non-null  object
 4   Mínima    10747 non-null  object
 5   Vol.      10747 non-null  object
 6   Var%      10747 non-null  object
dtypes: object(7)
memory usage: 587.9+ KB


In [3]:
# Convertendo a coluna 'Data' para o tipo datetime
df['Data'] = pd.to_datetime(df['Data'], errors='coerce', format='%d.%m.%Y')

# Eliminamos linhas com datas inválidas
df = df.dropna(subset=['Data'])

# Remove the header row which is still present as the first row
df = df.iloc[1:].reset_index(drop=True)

for col in ['Último', 'Abertura', 'Máxima', 'Mínima']:
    df[col] = df[col].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')


# Função para converter o volume textual (ex: 1.5B, 320M) para valores numéricos
def convert_large_numbers(value):
    if isinstance(value, str):
        value = value.strip().replace('.', '').replace(',', '.')
        if 'B' in value.upper():
            return float(value.upper().replace('B', '')) * 1_000_000_000
        elif 'M' in value.upper():
            return float(value.upper().replace('M', '')) * 1_000_000
        elif 'K' in value.upper():
            return float(value.upper().replace('K', '')) * 1_000
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# Apply the conversion function to 'Vol.' and 'Var%' columns
df['Vol.'] = df['Vol.'].apply(convert_large_numbers)
df['Var%'] = df['Var%'].astype(str).str.replace('%', '').apply(convert_large_numbers)


# Convertendo as colunas numéricas
for col in ['Último', 'Abertura', 'Máxima', 'Mínima']:
    df[col] = df[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.').astype(float)


# Removemos registros com valores ausentes nas colunas principais
df = df.dropna(subset=['Último', 'Vol.', 'Var%', 'Abertura', 'Máxima', 'Mínima']).reset_index(drop=True)

# Ordenamos as datas em ordem crescente
df = df.sort_values('Data').reset_index(drop=True)

df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,1982-01-07,0.0,0.0,0.0,0.0,116.0,9000000.0
1,1982-01-08,0.0,0.0,0.0,0.0,329.0,76000000.0
2,1982-01-11,0.0,0.0,0.0,0.0,127.0,52000000.0
3,1982-01-12,0.0,0.0,0.0,0.0,122.0,62000000.0
4,1982-01-13,0.0,0.0,0.0,0.0,106.0,19000000.0


In [4]:
df.tail()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
10739,2025-07-22,134036.0,134180.0,135300.0,133986.0,7.0,5.000000e+09
10740,2025-07-23,135368.0,134036.0,135782.0,133676.0,6.0,5.300000e+10
10741,2025-07-24,133808.0,135357.0,135363.0,133648.0,5.0,9.800000e+10
10742,2025-07-25,133524.0,133820.0,134204.0,133285.0,5.0,5.600000e+10
10743,2025-07-28,132129.0,133538.0,133902.0,131550.0,6.0,6.300000e+10


In [10]:
# Criamos novas colunas que serão usadas como variáveis para prever o comportamento do índice

df['Retorno_Dia'] = df['Último'].pct_change() * 100  # variação percentual diária
df['MM_5'] = df['Último'].rolling(window=5).mean()   # média móvel de 5 dias
df['MM_10'] = df['Último'].rolling(window=10).mean() # média móvel de 10 dias
df['MM_20'] = df['Último'].rolling(window=20).mean() # média móvel de 20 dias

# Preços de fechamento de dias anteriores (lags)
df['Lag_1'] = df['Último'].shift(1)
df['Lag_2'] = df['Último'].shift(2)
df['Lag_3'] = df['Último'].shift(3)

# Volume: médias móveis e variação percentual
df['Volume_MM_5'] = df['Vol.'].rolling(window=5).mean()
df['Volume_MM_10'] = df['Vol.'].rolling(window=10).mean()
df['Volume_Variacao'] = df['Vol.'].pct_change() * 100

# Target: 1 se o próximo fechamento for maior que o atual, senão 0
df['Target'] = (df['Último'].shift(-1) > df['Último']).astype(int)

# Removemos valores nulos resultantes das operações anteriores
df = df.dropna().reset_index(drop=True)
df.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%,Retorno_Dia,MM_5,MM_10,MM_20,Lag_1,Lag_2,Lag_3,Volume_MM_5,Volume_MM_10,Volume_Variacao,Target
0,1992-01-22,1.0,1.0,1.0,1.0,38.0,2.600000e+10,0.0,1.0,1.0,1.0,1.0,1.0,1.0,46.4,43.1,-29.629630,0
1,1992-01-23,1.0,1.0,1.0,1.0,95.0,5.200000e+10,0.0,1.0,1.0,1.0,1.0,1.0,1.0,52.6,49.7,150.000000,0
2,1992-01-24,1.0,1.0,1.0,1.0,84.0,1.000000e+10,0.0,1.0,1.0,1.0,1.0,1.0,1.0,63.0,54.8,-11.578947,0
3,1992-01-27,1.0,1.0,1.0,1.0,71.0,5.900000e+10,0.0,1.0,1.0,1.0,1.0,1.0,1.0,68.4,57.4,-15.476190,0
4,1992-01-28,1.0,1.0,1.0,1.0,57.0,5.300000e+10,0.0,1.0,1.0,1.0,1.0,1.0,1.0,69.0,57.9,-19.718310,0


In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8293 entries, 0 to 8292
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             8293 non-null   datetime64[ns]
 1   Último           8293 non-null   float64       
 2   Abertura         8293 non-null   float64       
 3   Máxima           8293 non-null   float64       
 4   Mínima           8293 non-null   float64       
 5   Vol.             8293 non-null   float64       
 6   Var%             8293 non-null   float64       
 7   Retorno_Dia      8293 non-null   float64       
 8   MM_5             8293 non-null   float64       
 9   MM_10            8293 non-null   float64       
 10  MM_20            8293 non-null   float64       
 11  Lag_1            8293 non-null   float64       
 12  Lag_2            8293 non-null   float64       
 13  Lag_3            8293 non-null   float64       
 14  Volume_MM_5      8293 non-null   float64

In [12]:
print("Tem NaN?", X.isna().sum().sum())
print("Tem Inf?", (~np.isfinite(X.to_numpy())).sum())
print("Tipos:", X.dtypes.unique())


Tem NaN? 0
Tem Inf? 0
Tipos: [dtype('float64')]


In [13]:
# Selecionamos as colunas que serão usadas como variáveis preditoras
features = [
    'Retorno_Dia', 'MM_5', 'MM_10', 'MM_20',
    'Lag_1', 'Lag_2', 'Lag_3',
    'Volume_MM_5', 'Volume_MM_10', 'Volume_Variacao'
]
X = df[features]
y = df['Target']

# 1. Seleciona as features
X = df[features].copy()
#Seleciona e força conversão de todas as features numéricas
X = df[features].apply(pd.to_numeric, errors='coerce')

# 2. Converte tudo para float (força conversão e erro se não puder)
X = X.astype(float)

# 3. Substitui infinitos por NaN
X.replace([float('inf'), float('-inf')], pd.NA, inplace=True)

# 4. Remove linhas com qualquer NaN
X = X.dropna()

# 5. Garante que y acompanhe os mesmos índices válidos
y = df.loc[X.index, 'Target']

# Definimos os últimos 30 registros como teste, e o restante como treino
# 1. Força a conversão das features para float e trata valores inválidos
X = df[features].apply(pd.to_numeric, errors='coerce')
X = X.replace([np.inf, -np.inf], np.nan)
X = X.dropna()

# 2. Alinha o target
y = df.loc[X.index, 'Target']

# 3. Agora sim: divide os dados
X_train = X[:-30]
X_test = X[-30:]
y_train = y[:-30]
y_test = y[-30:]

# Treinamos o modelo Random Forest
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

# Fazemos previsões com o modelo treinado
y_pred = modelo.predict(X_test)

# Avaliamos a performance do modelo
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acc:.2%}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


Acurácia: 66.67%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.68      0.95      0.79        20
           1       0.50      0.10      0.17        10

    accuracy                           0.67        30
   macro avg       0.59      0.53      0.48        30
weighted avg       0.62      0.67      0.58        30


Matriz de Confusão:
[[19  1]
 [ 9  1]]


### 🔧 Engenharia de Atributos Avançada + Novo Target + Tuning de Random Forest

In [14]:
# Novo Target: considerar subida mais significativa (ex: acima de 0.5%)
df['Target'] = (df['Retorno_Dia'].shift(-1) > 0.5).astype(int)

# Novas Features de Volatilidade e Comportamento
df['Volatilidade_5'] = df['Máxima'].rolling(5).std()
df['Shadow_Alta'] = df['Último'] - df['Mínima']
df['Acumulado_3d'] = df['Retorno_Dia'].rolling(3).sum()

# Dropa valores nulos criados
df.dropna(inplace=True)

# Features finais (incluindo as novas)
features = [
    'Retorno_Dia', 'MM_5', 'MM_10', 'MM_20',
    'Lag_1', 'Lag_2', 'Lag_3',
    'Volume_MM_5', 'Volume_MM_10', 'Volume_Variacao',
    'Volatilidade_5', 'Shadow_Alta', 'Acumulado_3d'
]

# Garantimos que todas sejam numéricas
import numpy as np
X = df[features].apply(pd.to_numeric, errors='coerce')
X = X.replace([np.inf, -np.inf], np.nan).dropna()
y = df.loc[X.index, 'Target']

# Divide treino e teste
X_train = X[:-30]
X_test = X[-30:]
y_train = y[:-30]
y_test = y[-30:]

# Modelo com tuning leve + balanceamento
from sklearn.ensemble import RandomForestClassifier
modelo = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    min_samples_split=10,
    class_weight='balanced',
    random_state=42
)
modelo.fit(X_train, y_train)

# Avaliação
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = modelo.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia com melhorias: {acc:.2%}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))


Acurácia com melhorias: 83.33%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.83      1.00      0.91        25
           1       0.00      0.00      0.00         5

    accuracy                           0.83        30
   macro avg       0.42      0.50      0.45        30
weighted avg       0.69      0.83      0.76        30


Matriz de Confusão:
[[25  0]
 [ 5  0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
